# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [4]:
from openai import OpenAI

client = OpenAI(base_url='https://aigptx.top/v1')

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一条木制的小路，蜿蜒穿过一片广阔的草地。草地绿意盎然，四周是茂密的植被与树木。天空清澈湛蓝，点缀着几朵轻盈的白云，给人一种宁静祥和的感觉。整个画面充满了自然的美景，阳光照射下，草地显得格外鲜艳。这样的小路仿佛通往未知的远方，给人一种探索和冒险的欲望，同时也让人感受到大自然的宁静与美好。', role='assistant', function_call=None, tool_calls=None))


In [5]:
response.choices[0].message.content

'这幅图展示了一条木制的小路，蜿蜒穿过一片广阔的草地。草地绿意盎然，四周是茂密的植被与树木。天空清澈湛蓝，点缀着几朵轻盈的白云，给人一种宁静祥和的感觉。整个画面充满了自然的美景，阳光照射下，草地显得格外鲜艳。这样的小路仿佛通往未知的远方，给人一种探索和冒险的欲望，同时也让人感受到大自然的宁静与美好。'

### 封装成一个函数 query_image_description

In [8]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(base_url='https://aigptx.top/v1')  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4o",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [10]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url, prompt='告诉我这图哪里搞笑了')
print(content)

这张图片的搞笑之处在于对比和夸张的表达方式。

左边的狗叫“16岁的我”，体格健壮，显示了年轻时充满希望和活力的状态，配文如“我前途一片光明”、“未来可期”、“我身体素质高”、“八九点钟的太阳”，都在表达年轻时的自信与理想。

右边的狗叫“工作后的我”，身形发胖或疲惫，显示了经历工作和生活后状态的巨大变化，配文如“好累好困”、“好想睡懒觉”、“我好弱小啊啊啊啊”、“重大伤病不要靠近我啊啊啊”、“我就是普通人”，这些台词都用夸张的方式体现了工作的压力和对生活的无奈，形成鲜明对比。

这种对比和反差往往会引起共鸣和幽默感，因为很多人可能在现实生活中也有类似的感受和经历，从而觉得这个对比非常搞笑。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [4]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(base_url='https://aigptx.top/v1')  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://aigptx.top/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [7]:
content = query_base64_image_description("./images/hand.jpg", prompt='基于手相学，分析图中的手相')
print(content)

手相学是一门古老的预测学科，通过观察手部的线条、形态和手指的特征来预测个体的性格、健康和未来。需要指出的是，手相学并没有科学依据，仅供娱乐和参考。

从这张手掌照片来看，可以观察到几条主要的掌纹，这些掌纹通常包括生命线、智慧线和感情线。以下是这些线的一般含义：

1. **生命线（Life Line）**：生命线是环绕拇指的那条弧形线，通常代表寿命、健康和重大生活变化。图中生命线比较明显，并且线条平缓，无明显中断，通常意味着健康状况良好，生命力强。

2. **智慧线（Head Line）**：智慧线是从拇指和食指之间开始，横向延伸的那条线。智慧线通常代表一个人的思想能力和决策能力。图中的智慧线比较长且清晰，一般认为这表明该人具有较强的思考能力和决策能力。

3. **感情线（Heart Line）**：感情线是从小指下方开始，横向穿过手掌上方的一条线，通常代表感情、爱情和人际关系。图中的感情线较为模糊且不明显，可能表明该人在感情方面较为含蓄或者不太容易表现出来。

除了以上三条主要的掌纹，还有一些其他特征也可以观察：

- **婚姻线**：位于小指下方，代表婚姻和亲密关系的状况。如果有多条线，可能代表多次恋爱或婚姻。
- **太阳线**：在无名指下方，代表名声和财富。如果太阳线清晰，可能预示着良好的事业运或财富运。

需要注意的是，手相学解释主要靠的是主观判断，仅供娱乐和参考，不应作为严肃的决策依据。如果有任何关于健康或生活方面的实际困扰，建议咨询相关专业人士。


#### 使用 GPT-4V 识别手写体笔记

![](./images/gpt-4v.jpg)

In [9]:
content = query_base64_image_description("./images/gpt-4v.jpg")
print(content)

这是一段描写自然景观的文字，内容为唐代文学家柳宗元的《小石潭记》。以下是全文解析：

----

**小石潭记  唐 柳宗元**

从小丘西行百二十步，隔篁竹，闻水声，如鸣珮环，心乐之。伐竹取道，下见小潭，水尤清冽。全石以为底，近岸卷石底以出，为坻，为屿，为嵁，为岩。青树翠蔓，蒙络摇缀，参差披拂。

潭中鱼可百许头，皆若空游无所依，日光下彻，影布石上，佁然不动；俶尔远逝，往来翕忽，似与游者相乐。

潭西南而望，斗折蛇行，明灭可见。其岸势犬牙差互，不可知其源。

坐潭上，四面竹树环合，寂寥无人，凄神寒骨，悄怆幽邃。以其境过清，不可久居，乃记之而去。

同游者：吴武陵，龚古，余弟宗玄。隶而从者，崔氏二小生。

----

**翻译：**

从小山丘向西走一百二十步，隔着竹林，听到水声，如同玉佩的声音，心里非常高兴。砍了竹子开道，向下发现一个小潭，水特别清澈。潭底全都是石头，靠近岸边的石头以卷曲的姿态露出水面，形成小沙洲、小岛屿、突出的石块和岩石。青树翠蔓，枝蔓交缠，错落有致地披散开来。潭中的鱼有一百多条，仿佛在空中游动，没有什么依凭。阳光穿透水面，影子映在石头上，静止不动；忽然之间，它们快速游远，来来回回，非常灵动，仿佛在与游人嬉戏。向西南方向望去，溪水曲折蜿蜒，时隐时现。河岸的地势犬牙交错，无法知道溪流的源头。坐在潭边，四周竹林树木环绕，寂静无人，令人心神凄凉，感到寒意入骨，惆怅幽深。因为此处环境过于清冷，不可久留，于是记下这段经历后离去。同游的人有：吴武陵，龚古，弟弟柳宗玄。随行的有：崔氏的两个小童。

---

这段文字描绘了一个小潭的美丽景色及作者游玩的感受，表达了柳宗元对大自然的热爱和对人文环境的某种深思。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [10]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这是一段描写自然景观的文字，内容为唐代文学家柳宗元的《小石潭记》。以下是全文解析：

----

**小石潭记  唐 柳宗元**

从小丘西行百二十步，隔篁竹，闻水声，如鸣珮环，心乐之。伐竹取道，下见小潭，水尤清冽。全石以为底，近岸卷石底以出，为坻，为屿，为嵁，为岩。青树翠蔓，蒙络摇缀，参差披拂。

潭中鱼可百许头，皆若空游无所依，日光下彻，影布石上，佁然不动；俶尔远逝，往来翕忽，似与游者相乐。

潭西南而望，斗折蛇行，明灭可见。其岸势犬牙差互，不可知其源。

坐潭上，四面竹树环合，寂寥无人，凄神寒骨，悄怆幽邃。以其境过清，不可久居，乃记之而去。

同游者：吴武陵，龚古，余弟宗玄。隶而从者，崔氏二小生。

----

**翻译：**

从小山丘向西走一百二十步，隔着竹林，听到水声，如同玉佩的声音，心里非常高兴。砍了竹子开道，向下发现一个小潭，水特别清澈。潭底全都是石头，靠近岸边的石头以卷曲的姿态露出水面，形成小沙洲、小岛屿、突出的石块和岩石。青树翠蔓，枝蔓交缠，错落有致地披散开来。潭中的鱼有一百多条，仿佛在空中游动，没有什么依凭。阳光穿透水面，影子映在石头上，静止不动；忽然之间，它们快速游远，来来回回，非常灵动，仿佛在与游人嬉戏。向西南方向望去，溪水曲折蜿蜒，时隐时现。河岸的地势犬牙交错，无法知道溪流的源头。坐在潭边，四周竹林树木环绕，寂静无人，令人心神凄凉，感到寒意入骨，惆怅幽深。因为此处环境过于清冷，不可久留，于是记下这段经历后离去。同游的人有：吴武陵，龚古，弟弟柳宗玄。随行的有：崔氏的两个小童。

---

这段文字描绘了一个小潭的美丽景色及作者游玩的感受，表达了柳宗元对大自然的热爱和对人文环境的某种深思。

![](./images/handwriting_1.jpg)

In [12]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示的是一本笔记本的两页，内容涉及深度学习、特别是关于自然语言处理（NLP）的各种技术和方法。主要讨论了Transformer模型及其改进方法和训练技术。

左侧页面的上部标注有“自然语言处理”、“基础”和“评价”，可能是对内容的分类。提到了Transformer模型，并列举了不同的测试标准和指标，如PeFT (“Prompt-based Fine-Tuning”) 和模型性能对比（“Benchmark”）。此外，还提到了不同的方法，如Prompt Tuning和Adapter。具体包括：  
- Adapter: 一个2019年Google的研究
- Prefix: 代表2021年Stanford的工作
- Prompt: 同样是2021年Google的研究
- P-Tuning V1和V2：2021年的两种方法
- Soft prompts：2021年的研究，提示模板基于模板

右侧页面讨论了多模态指令式微调（multi-modality instruction FT）、Llama (3B)、LoRA、PETC（2022年的新技术）等。还有部分文字描述了如何使用prefix-tuning和Adapter方法来细化在大型语言模型（LLMs）中的处理。

页面提到了几种语言模型，如：
- Llama 
- BLOOM
- ChatGLM 
- Alpaca

这些内容表明这本笔记本的主人正在研究或学习NLP领域的最新技术和方法，特别是如何通过各种微调技术提升已有的大型语言模型的性能。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

![](./images/gpt-4v.jpg)

In [8]:
from openai import OpenAI
import base64
import requests
import json
from IPython.display import display, Markdown

client = OpenAI(base_url='https://aigptx.top/v1')  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://aigptx.top/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return Markdown(content)
    else:
        return f"Error: {response.status_code}, {response.text}"

In [9]:
res = query_base64_image_description("./images/gpt-4v.jpg")
display(res)

这是一篇手写的古文，文章内容如下：

```
役小丘西行百二十步隔篁竹闻水声如鸣珮环心乐之伐竹取道下见小潭水尤清冽全石以为底近岸卷石底以出为坻为嵁为岩青树翠蔓蒙络摇缀参差披拂潭中鱼可百许头皆若空游无所依日光下澈影布石上佁然不动俶尔远逝往来翕忽如怵然惊折跃犹可见其脍与游者相乐潭西南而望斗折蛇行明灭可见其脰潭上四面竹树环合寂寥无人凄神寒骨悄怆幽邃以为凡是州之山有异态者皆我有也而未始知西山之怪特无同游者吴武陵醉翁亭记余幼好书古余常游郡城池森林古寺每欲依高僧为飡饭图史消永昼今号怀古余宗言披宗而求之众表而役者凭崖小石潭记

知止书
```

这篇文章为柳宗元的《小石潭记》，描述了作者与朋友游览衡山县西南的小石潭的经历。文章从行进路线、潭水的清澈、石底景象、鱼的活动、周围的植物和环境的寂静等多方面生动地描绘了小石潭的自然景色，表达了作者对自然美景的喜爱及心境的清幽。文章意境优美，文字优雅，是古文中的经典作品之一。